In [1]:
import math

In [2]:
def fast_exp(base, exponent, modulus):
  result = 1
  while exponent != 0:
    if exponent % 2 == 1:
      result = (result * base) % modulus
    base = base**2 % modulus
    exponent = exponent // 2
  return result

def extended_euclidean(m, n):
  if n == 0:
    return 1, 0, m
  x, y, g = extended_euclidean(n, m % n)
  return y, x - (m // n)*y, g

def inv_mod(a, modulus):
  s, _, g = extended_euclidean(a, modulus)
  assert g == 1, ValueError('the modular inverse does not exist')
  return s % modulus

def crt(remainders, moduli):
  assert len(remainders) == len(moduli), ValueError('the lists remainders and moduli are not the same length')
  assert len(remainders) > 0, ValueError('the list lengths must be greater than one')

  first_modulus = moduli[0]
  first_remainder = remainders[0]

  if len(remainders) == 1:
    return first_remainder % first_modulus, first_modulus
  
  consecutive_remainder, consecutive_modulus = crt(remainders[1:], moduli[1:])
  u, v, g = extended_euclidean(consecutive_modulus, first_modulus)

  assert g == 1, ValueError('the moduli are not relatively prime')

  return (first_remainder*consecutive_modulus*u + consecutive_remainder*first_modulus*v) % (first_modulus*consecutive_modulus), first_modulus*consecutive_modulus

def polynomial_root(exponent, a, modulus):
  u = inv_mod(exponent, modulus - 1)

  result = fast_exp(a, u, modulus)

  return result

def cf_float(x, n):
  cf = []
  for _ in range(n):
    cf.append(int(x))
    x = 1 / (x % 1)
  return cf

def continued_frac(num, den):
  cf = []
  while den != 0:
    cf.append(num//den)
    num, den = den, num % den
  return cf

def get_convergents(cf):
  convergents = [[0, 1], [1, 0]]
  for a in cf:
    hk2, hk1 = convergents[-2:]
    convergents.append([a*hk1[0] + hk2[0], a*hk1[1] + hk2[1]])
  return convergents[2:]

def crack_rsa(e, n):
  convergents = get_convergents(continued_frac(e, n))
  for convergent in convergents:
    k, d = convergent
    if k != 0 and (phi := (d * e - 1) // k) != 0:
      p_q = n - phi + 1
      disc = p_q**2 - 4*n
      if disc > 0:
        p, q = (p_q + math.isqrt(disc)) // 2, (p_q - math.isqrt(disc)) // 2
        if p*q == n:
          return d, p, q
  return -1, -1, -1

def pollard_rho(n):
  f = lambda x_: x_**2 + 1
  x = 2
  y = 2
  g = 1
  count = 0
  while g == 1:
    x = f(x) % n
    y = f(f(y)) % n
    _, _, g = extended_euclidean(abs(x-y), n)
    count += 1
  return g, count

In [3]:
def print_cycles(n):
  h = lambda x_: x_**2 + 1
  x = 2
  for _ in range(n):
    x = h(x) % n
    print(x)

In [5]:
pollard_rho(720777718430892426412699013841273041436894957841223)

(198765432101, 228360)

In [6]:
pollard_rho(720777718430892426412699013841273041436894957841223//198765432101)

(18826507658281, 1544523)

In [7]:
crt([8187081806215505471, 5285356194805239972, 6399213745575232470], [8876044532898802067, 18801105946394093459, 39199007823998693533])

(1881676371789154860897069,
 6541509009210323465565388766012318656540490492288685237349)

In [8]:
1881676371789154860897069**(1/3)

123456788.99999987

In [9]:
123456789**3

1881676371789154860897069

In [10]:
extended_euclidean(65540, 65537)

(21846, -21847, 1)

In [14]:
inv_mod(1622980102927711263837825272257800131879, 2169657800746705868906997566280821497741)

1686884397782746217165356841165126481871

In [15]:
fast_exp(1147836106621857253994178866934024626264, 21846, 2169657800746705868906997566280821497741)

218938390921748345651315330467865881255

In [17]:
fast_exp(1686884397782746217165356841165126481871, 21847, 2169657800746705868906997566280821497741)

1465484096981562922823936098994504215990

In [18]:
(218938390921748345651315330467865881255 * 1465484096981562922823936098994504215990) % 2169657800746705868906997566280821497741

2718281828